In [1]:
import pandas as pd
import numpy as np
import os
from pymining import seqmining, itemmining

In [2]:
BASE_DIR = "fixationpoints/"
OUT_DIR = "output/"
CUTOFF_TIME = 5000

In [3]:
if not os.path.exists(os.path.join(BASE_DIR,OUT_DIR)):
    os.mkdir(os.path.join(BASE_DIR,OUT_DIR))     

In [24]:
class GeneratePattern:
    def __init__(self):
        pass
    
    def read_data(self,file):
        data = np.array(pd.read_csv(file))
        proc = []
        for row in data:
            if row[1] != 0:
                proc.append([row[1],row[3],row[6],row[7]])
        return np.array(proc)
    
    def gen_lines_array(self,data):
        ans = []
        curr_line = data[0][0]
        ln = data.shape[0]
        ans.append(data[0][0])
        for i in range(1,ln):
            if curr_line != data[i][0]:
                ans.append(data[i][0])
                curr_line = data[i][0]
            
        return ans
            

    def gen_pattern_string(self,data):
        pats = []
        ans = ""
        ln = data.shape[0]
        curr_line = data[0][0]
        curr_token = data[0][1]
        start_time = data[0][2]
        
        for i in range(1,ln):
            if data[i][2] - start_time > CUTOFF_TIME:
                if ans != "":
                    pats.append(ans)
                ans = ""
                start_time = data[i][2]
            if data[i][0] == curr_line:
                if data[i][1] > curr_token:
                    ans += "E"
                elif data[i][1] < curr_token:
                    ans += "F"
            elif data[i][0] > curr_line:
                if data[i][0] == curr_line + 1:
                    ans += "A"
                else:
                    pass
                    #ans += "B"
            else:
                if data[i][0] + 1 == curr_line:
                    ans += "C"
                else:
                    pass
                    #ans += "D"
            curr_line = data[i][0]
            curr_token = data[i][1]
            
        if ans != "":
            pats.append(ans)
        return pats
    
    def generate_patterns_array(self,patterns):
        transactions = []
        for cand, pat in enumerate(patterns):
            ln = len(pat)
            curr = []
            for i in range(ln):
                curr.append((pat[i]))
            
            transactions.append(curr)
        return transactions
    
    def generate_pattern_file(self,patterns,filename):
        with open(filename,'w') as f:
            num_cand = len(patterns)
            for cand, pat in enumerate(patterns):
                ln = len(pat)
                for i in range(ln):
                    f.write(str(cand+1)+" "+str(i+1)+" "+pat[i])
                    if cand==num_cand-1 and i == ln-2:
                        pass
                    else:
                        f.write("\n")
            
    

## Pattern Chars - 
A - Vertical Next<br>
B - Vertical Later<br>
C - Vertical Previous<br>
D - Vertical Earlier<br>
E - Horizontal Later<br>
F - Horizontal Earlier<br>

In [18]:
pats = []
fsegs = []
for file in os.listdir(os.path.join(BASE_DIR,"5")):
    gp = GeneratePattern()
    data = gp.read_data(os.path.join(BASE_DIR,"5",file))
    pat = gp.gen_pattern_string(data)
    print(pat)
    fsegs.append(seqmining.freq_seq_enum(pat,2))
    pats.extend(pat)
trans = gp.generate_patterns_array(pats)
gp.generate_pattern_file(pats,"SPAM_5_single.in")

['FFEEEFEE', 'EFEEEFFEFE', 'FACEFEEEFACE', 'FFACAAFEF', 'EF', 'EEFEFEFFFE', 'C', 'FEFE', 'FEE', 'AEF', 'EAF', 'ECEF', 'ECFEFE', 'C', 'CF']
['AA', 'E', 'C']
['F', 'CEFAEA']
['A', 'EFAE', 'FACFE', 'EFFAECF', 'A', 'CEEAEFF', 'EEFF', 'EEFCC', 'FFA', 'FEF', 'CAAAE', 'EEACA', 'CF']
['CACEEA', 'EAFAACACA', 'EEFE', 'FAF', 'CAC', 'AAACFEF', 'CAACFCA', 'EF', 'CAFEEFACC', 'AACF', 'CCACFAACAC', 'FA', 'EF']
['CF', 'EE', 'E', 'E', 'A']
['CEEAEEECF', 'FFEEEEECEF', 'AECEE', 'FCFACE', 'CAACAECEAEF', 'ACACEFE', 'EAECCFF', 'A', 'CCAE', 'E', 'E', 'FAAFE', 'ACFE', 'CCAF', 'CA']
['FEE', 'EFEEEAEEEFFE', 'ACFEFFA', 'FEEF', 'CCFC', 'FEAAC', 'CAEFC', 'EEACEAFEEF', 'FACAECAFEFF', 'ACAFEACEEAFEECA', 'EFCAEEFEE', 'FAFCEAFEC', 'CFFFE', 'FCFAFFCACEEAEC', 'EFACAEF', 'AFECAECFA', 'CCEF', 'CFAFAC', 'CFEFAFAC', 'AAFCAA', 'AAFFCAFEEE', 'ACAEFECAFCAEF', 'EFFCAFFEEEEFECA', 'FAACEEAEFEE', 'FFEACFACAEEF', 'EFFEAAAE', 'FAA']


In [25]:
line_pats = []
for file in os.listdir(os.path.join(BASE_DIR,"5")):
    gp = GeneratePattern()
    data = gp.read_data(os.path.join(BASE_DIR,"5",file))
    line_pat = gp.gen_lines_array(data)
    line_pats.append(line_pat)

In [19]:
cts = {}
for seg in fsegs:
    for key, val in seg:
        if key not in cts:
            cts[key] = 0
        cts[key] += val
        #print(key,val)

In [23]:
srt = sorted(fsegs[0],key=lambda x:x[1],reverse=True )
srt

[(('F',), 13),
 (('E',), 12),
 (('E', 'F'), 11),
 (('F', 'E'), 8),
 (('E', 'E'), 8),
 (('F', 'F'), 7),
 (('F', 'F', 'E'), 7),
 (('F', 'E', 'F'), 7),
 (('F', 'E', 'E'), 7),
 (('C',), 7),
 (('F', 'E', 'F', 'E'), 6),
 (('E', 'E', 'F'), 6),
 (('E', 'F', 'E'), 6),
 (('F', 'F', 'F'), 5),
 (('F', 'F', 'F', 'E'), 5),
 (('F', 'F', 'E', 'F'), 5),
 (('E', 'E', 'F', 'E'), 5),
 (('C', 'F'), 5),
 (('E', 'E', 'E'), 5),
 (('E', 'F', 'E', 'E'), 5),
 (('F', 'F', 'E', 'F', 'E'), 4),
 (('E', 'F', 'E', 'F', 'E'), 4),
 (('C', 'E'), 4),
 (('E', 'E', 'E', 'F'), 4),
 (('E', 'E', 'E', 'E'), 4),
 (('F', 'E', 'E', 'F', 'E'), 4),
 (('F', 'E', 'E', 'E'), 4),
 (('E', 'F', 'F', 'E'), 4),
 (('F', 'E', 'E', 'F'), 4),
 (('A', 'F'), 4),
 (('E', 'F', 'E', 'F'), 4),
 (('E', 'E', 'E', 'F', 'E'), 4),
 (('E', 'E', 'E', 'E', 'E'), 4),
 (('A',), 4),
 (('E', 'F', 'F'), 4),
 (('F', 'E', 'F', 'E', 'E'), 4),
 (('C', 'E', 'F'), 4),
 (('F', 'F', 'E', 'E'), 4),
 (('A', 'E'), 3),
 (('F', 'E', 'E', 'E', 'E'), 3),
 (('F', 'E', 'E', 'E', 

In [8]:
data.shape

(392, 2)

In [9]:
ans

'FEBEBEFEEEAEEEFFEACBFEFFABDFBEDEFDCBDCFDCBFDBEABACDCAEFDBCEEACEAFEEFFACAECAFEFDFACAFEACEEAFEECAEFCAEEFBDEEBFAFCEAFECBDCDFDBFFEFCFAFFCACEEAECEFACAEFBBADBDFECAECFABDCCBBEFDCFAFDABBCDCFEFAFACABDAFCADAADAFFCAFEEEACAEFECAFCAEFEFFCAFFEEEEFECADFAACEEAEFBEEFFEACFACAEEBFBDEFFEBAAAEDFBDBBAABBDB'

In [10]:
gp.generate_pattern_file([ans,ans],"sample_out.in")

In [75]:
freq_seqs = seqmining.freq_seq_enum(pats,8)

In [76]:
freq_seqs

{(('A',), 60),
 (('A', 'A'), 35),
 (('A', 'A', 'A'), 17),
 (('A', 'A', 'A', 'E'), 12),
 (('A', 'A', 'A', 'E', 'F'), 8),
 (('A', 'A', 'A', 'F'), 9),
 (('A', 'A', 'B'), 13),
 (('A', 'A', 'C'), 22),
 (('A', 'A', 'C', 'A'), 13),
 (('A', 'A', 'C', 'A', 'E'), 8),
 (('A', 'A', 'C', 'C'), 8),
 (('A', 'A', 'C', 'E'), 11),
 (('A', 'A', 'C', 'E', 'F'), 8),
 (('A', 'A', 'C', 'F'), 12),
 (('A', 'A', 'D'), 13),
 (('A', 'A', 'E'), 19),
 (('A', 'A', 'E', 'E'), 10),
 (('A', 'A', 'E', 'F'), 12),
 (('A', 'A', 'F'), 18),
 (('A', 'A', 'F', 'E'), 11),
 (('A', 'B'), 32),
 (('A', 'B', 'B'), 10),
 (('A', 'B', 'D'), 14),
 (('A', 'C'), 38),
 (('A', 'C', 'A'), 23),
 (('A', 'C', 'A', 'A'), 11),
 (('A', 'C', 'A', 'C'), 12),
 (('A', 'C', 'A', 'C', 'A'), 9),
 (('A', 'C', 'A', 'C', 'E'), 8),
 (('A', 'C', 'A', 'E'), 15),
 (('A', 'C', 'A', 'E', 'E'), 10),
 (('A', 'C', 'A', 'E', 'F'), 11),
 (('A', 'C', 'A', 'F'), 13),
 (('A', 'C', 'A', 'F', 'E'), 9),
 (('A', 'C', 'B'), 10),
 (('A', 'C', 'C'), 15),
 (('A', 'C', 'C', 'A'),

In [13]:
ans

'FEBEBEFEEEAEEEFFEACBFEFFABDFBEDEFDCBDCFDCBFDBEABACDCAEFDBCEEACEAFEEFFACAECAFEFDFACAFEACEEAFEECAEFCAEEFBDEEBFAFCEAFECBDCDFDBFFEFCFAFFCACEEAECEFACAEFBBADBDFECAECFABDCCBBEFDCFAFDABBCDCFEFAFACABDAFCADAADAFFCAFEEEACAEFECAFCAEFEFFCAFFEEEEFECADFAACEEAEFBEEFFEACFACAEEBFBDEFFEBAAAEDFBDBBAABBDB'

In [77]:
fs = sorted(freq_seqs,key=lambda x: (x[1],len(x[0])),reverse=True)

In [78]:
fs

[(('B',), 84),
 (('D',), 73),
 (('F',), 72),
 (('E',), 68),
 (('A',), 60),
 (('C',), 58),
 (('B', 'D'), 53),
 (('E', 'F'), 47),
 (('D', 'B'), 46),
 (('C', 'F'), 44),
 (('F', 'E'), 42),
 (('E', 'E'), 42),
 (('F', 'F'), 40),
 (('A', 'F'), 40),
 (('A', 'E'), 38),
 (('D', 'D'), 38),
 (('F', 'B'), 38),
 (('B', 'B'), 38),
 (('A', 'C'), 38),
 (('C', 'A'), 37),
 (('C', 'E'), 37),
 (('F', 'A'), 35),
 (('A', 'A'), 35),
 (('B', 'E'), 33),
 (('B', 'F'), 32),
 (('E', 'B'), 32),
 (('A', 'B'), 32),
 (('D', 'F'), 31),
 (('F', 'D'), 30),
 (('E', 'A'), 30),
 (('D', 'E'), 29),
 (('A', 'D'), 29),
 (('F', 'C'), 29),
 (('E', 'E', 'F'), 28),
 (('C', 'C'), 28),
 (('F', 'E', 'F'), 27),
 (('C', 'E', 'F'), 27),
 (('B', 'C'), 27),
 (('E', 'C'), 27),
 (('D', 'C'), 27),
 (('C', 'B'), 27),
 (('C', 'D'), 27),
 (('C', 'A', 'E'), 26),
 (('F', 'F', 'E'), 25),
 (('F', 'E', 'E'), 25),
 (('A', 'E', 'F'), 25),
 (('C', 'A', 'F'), 25),
 (('D', 'A'), 25),
 (('B', 'D', 'B'), 24),
 (('C', 'F', 'E'), 24),
 (('A', 'C', 'F'), 24),


In [56]:
from prefixspan import PrefixSpan

In [79]:
db = [
    [0, 1, 3, 4,5],
    [1, 1, 1, 3, 4],
    [2, 1, 2, 2, 0],
    [1, 1, 1, 2, 2],
]

ps = PrefixSpan(pats)

In [80]:
print(ps.frequent(8,closed=True))

[(72, ['F']), (40, ['F', 'F']), (25, ['F', 'F', 'E']), (14, ['F', 'F', 'E', 'E']), (12, ['F', 'F', 'E', 'F']), (8, ['F', 'F', 'E', 'C']), (18, ['F', 'F', 'F']), (12, ['F', 'F', 'F', 'E']), (14, ['F', 'F', 'B']), (8, ['F', 'F', 'D']), (18, ['F', 'F', 'A']), (11, ['F', 'F', 'A', 'C']), (11, ['F', 'F', 'A', 'E']), (9, ['F', 'F', 'A', 'F']), (15, ['F', 'F', 'C']), (10, ['F', 'F', 'C', 'E']), (8, ['F', 'F', 'C', 'A']), (8, ['F', 'F', 'C', 'F']), (42, ['F', 'E']), (25, ['F', 'E', 'E']), (13, ['F', 'E', 'E', 'E']), (9, ['F', 'E', 'E', 'E', 'F']), (9, ['F', 'E', 'E', 'E', 'E']), (8, ['F', 'E', 'E', 'E', 'E', 'E']), (17, ['F', 'E', 'E', 'F']), (9, ['F', 'E', 'E', 'F', 'E']), (8, ['F', 'E', 'E', 'A']), (8, ['F', 'E', 'E', 'C']), (27, ['F', 'E', 'F']), (15, ['F', 'E', 'F', 'E']), (8, ['F', 'E', 'F', 'E', 'E']), (8, ['F', 'E', 'F', 'B']), (9, ['F', 'E', 'F', 'F']), (9, ['F', 'E', 'F', 'A']), (8, ['F', 'E', 'F', 'C']), (20, ['F', 'E', 'B']), (9, ['F', 'E', 'B', 'E']), (8, ['F', 'E', 'B', 'D']), (12

In [59]:
print(ps.frequent(2,closed=True))

[(2, [0]), (4, [1]), (3, [1, 2]), (2, [1, 2, 2]), (2, [1, 3, 4]), (2, [1, 1, 1])]


In [64]:
print(ps.frequent(2,generator=True))

[(2, [0]), (2, [1, 1]), (2, [3]), (2, [4]), (2, [2])]


In [9]:
stct = sorted(cts.items(),key = lambda x:x[1],reverse=True)

In [10]:
stct

[(('B',), 84),
 (('D',), 73),
 (('F',), 71),
 (('E',), 66),
 (('A',), 57),
 (('C',), 55),
 (('B', 'D'), 53),
 (('E', 'F'), 46),
 (('D', 'B'), 44),
 (('C', 'F'), 42),
 (('F', 'E'), 41),
 (('F', 'F'), 40),
 (('A', 'F'), 40),
 (('E', 'E'), 40),
 (('A', 'C'), 38),
 (('B', 'B'), 37),
 (('F', 'B'), 37),
 (('D', 'D'), 37),
 (('A', 'E'), 37),
 (('C', 'E'), 36),
 (('C', 'A'), 36),
 (('F', 'A'), 34),
 (('B', 'E'), 33),
 (('A', 'A'), 33),
 (('B', 'F'), 31),
 (('E', 'B'), 30),
 (('F', 'D'), 30),
 (('D', 'F'), 29),
 (('E', 'A'), 29),
 (('F', 'C'), 29),
 (('D', 'E'), 28),
 (('A', 'B'), 28),
 (('E', 'E', 'F'), 28),
 (('E', 'C'), 27),
 (('F', 'E', 'F'), 26),
 (('C', 'C'), 26),
 (('A', 'E', 'F'), 25),
 (('C', 'D'), 25),
 (('C', 'E', 'F'), 25),
 (('D', 'C'), 25),
 (('C', 'A', 'E'), 25),
 (('F', 'F', 'E'), 25),
 (('A', 'D'), 25),
 (('C', 'B'), 24),
 (('A', 'C', 'F'), 24),
 (('B', 'C'), 24),
 (('F', 'E', 'E'), 23),
 (('B', 'D', 'B'), 23),
 (('B', 'A'), 23),
 (('C', 'A', 'F'), 23),
 (('C', 'F', 'E'), 22),


In [35]:
line_seqs = seqmining.freq_seq_enum(line_pats,6)

In [36]:
srt = [x[0] for x in line_seqs]
srt = sorted(srt,key=lambda x:len(x),reverse=True)
srt

[(7, 6, 8, 8, 8, 7, 11, 10, 17, 17),
 (7, 6, 10, 8, 8, 8, 11, 10, 17, 17),
 (1, 7, 6, 8, 10, 8, 6, 8, 8, 7),
 (7, 6, 10, 8, 8, 7, 11, 10, 17, 17),
 (7, 6, 8, 10, 8, 7, 11, 10, 17, 17),
 (7, 6, 8, 10, 8, 6, 11, 10, 17, 17),
 (6, 10, 8, 8, 8, 11, 10, 17, 17),
 (7, 6, 10, 8, 8, 11, 10, 17, 17),
 (7, 6, 10, 8, 8, 7, 11, 17, 17),
 (7, 8, 10, 8, 7, 11, 10, 17, 17),
 (7, 6, 8, 8, 8, 7, 11, 10, 17),
 (1, 7, 6, 8, 10, 8, 8, 8, 7),
 (7, 6, 8, 10, 6, 11, 10, 17, 17),
 (7, 6, 8, 8, 8, 11, 10, 17, 17),
 (11, 17, 7, 6, 8, 10, 8, 6, 11),
 (1, 7, 6, 8, 10, 8, 6, 8, 7),
 (1, 11, 7, 8, 10, 8, 6, 8, 8),
 (6, 8, 10, 8, 6, 11, 10, 17, 17),
 (1, 7, 6, 10, 8, 8, 8, 7, 11),
 (1, 7, 6, 10, 8, 6, 8, 8, 7),
 (7, 6, 8, 8, 8, 7, 11, 17, 17),
 (11, 17, 7, 6, 8, 10, 8, 11, 10),
 (1, 7, 6, 8, 10, 6, 8, 8, 7),
 (7, 6, 8, 10, 8, 6, 11, 10, 17),
 (1, 7, 6, 8, 10, 8, 6, 8, 8),
 (1, 7, 8, 10, 8, 6, 8, 8, 7),
 (1, 7, 6, 8, 10, 8, 6, 19, 7),
 (7, 6, 8, 10, 8, 6, 11, 17, 17),
 (7, 6, 8, 8, 6, 11, 10, 17, 17),
 (7, 8, 10, 8, 

In [29]:
for pat in line_pats:
    print(pat)

[1, 8, 4, 6, 7, 6, 7, 6, 7, 6, 7, 8, 10, 8, 7, 10, 6, 8, 11, 13, 17, 13, 17, 19, 17, 7, 8, 10, 11, 8, 10, 13, 17, 11, 10, 8, 10, 7, 6, 13, 11, 10, 8, 10, 13, 10, 8, 11, 10, 13, 19, 1, 17, 6, 11]
[11, 16, 17, 8, 6, 7, 17, 20, 19, 17, 11, 19, 17, 8]
[1, 7, 1, 19, 17, 11, 10, 11, 7, 8, 16, 6, 10]
[1, 2, 6, 7, 10, 11, 10, 11, 7, 17, 16, 17, 19, 16, 7, 6, 7, 10, 13, 10, 8, 13, 8, 13, 10, 8, 7, 6, 10, 13, 10, 11, 13, 10, 13, 10, 13, 17, 16, 17, 14, 16, 17, 15, 16, 17, 16, 17, 19, 8, 10, 8, 7]
[7, 6, 7, 6, 7, 8, 9, 10, 8, 7, 8, 7, 8, 11, 6, 4, 11, 6, 8, 10, 11, 15, 17, 16, 17, 7, 6, 4, 1, 6, 7, 8, 6, 7, 6, 11, 10, 11, 12, 11, 10, 7, 8, 14, 11, 7, 6, 7, 8, 7, 6, 7, 8, 7, 10, 8, 7, 6, 7, 6, 7, 8, 7, 8, 7, 2, 8, 6, 7, 19, 8, 6, 18, 16, 4, 11, 20, 7]
[1, 11, 17, 7, 2, 1, 6, 8, 10, 8, 6, 8, 1, 17, 8, 1, 19, 1, 7, 13, 11, 1, 2, 10, 17, 13, 17]
[2, 1, 6, 7, 6, 8, 10, 17, 16, 10, 8, 10, 11, 10, 7, 10, 17, 16, 17, 8, 7, 6, 8, 6, 7, 8, 7, 8, 7, 8, 10, 11, 10, 11, 10, 13, 11, 6, 10, 8, 11, 7, 8, 7, 6, 1